In [2]:
!pip install llama-index-vector-stores-qdrant==0.3.0 -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### 3.4.1 라마인덱스 기반 답변 생성

In [3]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from llama_index.core.schema import Document

In [4]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")
# 문서 데이터
documents = [
    "고양이는 작은 육식동물로, 주로 애완동물로 기릅니다. 민첩하고 장난기 있는 행동으로 유명합니다.",
    "강아지는 충성심이 강하고 친절한 동물로, 흔히 인간의 최고의 친구로 불립니다. 주로 애완동물로 기르고, 동반자로서 유명합니다.",
    "고양이와 강아지는 전 세계적으로 인기 있는 애완동물로, 각각 독특한 특징을 가지고 있습니다."
]
ids = ["doc1", "doc2", "doc3"]

# 문서를 Document 형식으로 변환
documents = [Document(text=doc, id_=doc_id) for doc, doc_id in zip(documents, ids)]

In [5]:
# 특정 디렉토리에 저장 할 경우
client = QdrantClient(path="./qdrant_db")

In [10]:
collections = client.get_collections()
collection_list = [collection.name for collection in collections.collections]

In [16]:
# Qdrant에 새로운 컬렉션 생성
collection_name = "llama_index_qdrant"

if collection_name not in collection_list:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=768, distance=Distance.COSINE)
    )

In [19]:
# Qdrant 벡터 스토어 설정
vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
# 인덱스 생성 및 저장
index = VectorStoreIndex.from_documents(documents, vector_store=vector_store)

In [20]:
# 쿼리 엔진 생성 및 실행
query_engine = index.as_query_engine()
query_text = "고양이"
response = query_engine.query(query_text)

# 최종 응답 출력
print("[질의 결과]")
print(response)

# 응답 생성에 사용된 문서 확인
print("\n[응답에 사용된 문서]")
for i, node in enumerate(response.source_nodes, 1):
    print(f"{i}. {node.text}\n")

[질의 결과]
고양이는 작은 육식동물로, 주로 애완동물로 기르며 민첩하고 장난기 있는 행동으로 유명합니다.

[응답에 사용된 문서]
1. 고양이는 작은 육식동물로, 주로 애완동물로 기릅니다. 민첩하고 장난기 있는 행동으로 유명합니다.

2. 고양이와 강아지는 전 세계적으로 인기 있는 애완동물로, 각각 독특한 특징을 가지고 있습니다.



### 3.4.3 클라우드 기반 환경 설정

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from llama_index.core.schema import Document

# Qdrant 클라우드 서버에 연결 (API 키 사용)
QDRANT_API_KEY = "your_api_key"
QDRANT_HOST = "your_host:6333"
client = QdrantClient(QDRANT_HOST, api_key=QDRANT_API_KEY)

In [28]:
# Qdrant 벡터 스토어 설정
vector_store = QdrantVectorStore(client=client, collection_name=collection_name)

# 인덱스 생성 및 저장
index = VectorStoreIndex.from_documents(documents, vector_store=vector_store)

# 쿼리 엔진 생성 및 실행
query_engine = index.as_query_engine()
query_text = "고양이"
response = query_engine.query(query_text)

# 최종 응답 출력
print("\n[질의 결과]")
print(response)

# 응답 생성에 사용된 문서 확인
print("\n[응답에 사용된 문서]")
for i, node in enumerate(response.source_nodes, 1):
    print(f"{i}. {node.text}\n")


[질의 결과]
고양이는 작은 육식동물로, 주로 애완동물로 기르며 민첩하고 장난기 있는 행동으로 유명합니다.

[응답에 사용된 문서]
1. 고양이는 작은 육식동물로, 주로 애완동물로 기릅니다. 민첩하고 장난기 있는 행동으로 유명합니다.

2. 고양이와 강아지는 전 세계적으로 인기 있는 애완동물로, 각각 독특한 특징을 가지고 있습니다.

